In [1]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import BaseEstimator
from sklearn.datasets import fetch_openml, fetch_20newsgroups

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

from scipy.spatial import distance
from sklearn.neighbors import KDTree
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold

##### Задание 1 (1 балл)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. 
Для подсчета расстояний можно использовать функции [отсюда](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html)

In [2]:
class MyKNeighborsClassifier(BaseEstimator):
    
    def __init__(self, n_neighbors = 2, algorithm='brute', metric='euclidean'):
        self.n_neighbors = n_neighbors
        self.algorithm = algorithm
        self.metric = metric
        
        self.X_train = None
        self.y_train = None
    
    def dist(self, x, x_i):
        return np.sqrt(np.sum((x - x_i) ** 2, axis=1))

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
    
    def predict(self, X):
        if self.algorithm == 'brute':
            dist = distance.cdist(self.X_train, X, metric=self.metric)
            indices = np.argpartition(dist, self.n_neighbors, axis=0)[:self.n_neighbors]
            test = self.y_train[indices].transpose()
            res = np.ndarray((0,), dtype=int)
            for x in test:
                res = np.append(res, np.argmax(np.bincount(x)))
            return res
        if self.algorithm == 'kd_tree':
            res = np.ndarray((0,), dtype=int)
            for x in X:
                tmp = np.append(np.ravel(self.X_train), x).reshape((-1, self.X_train.shape[1]))
                test = self.y_train[KDTree(tmp[:-1], metric=self.metric).query(tmp[-1:], k=self.n_neighbors)[1]].ravel()
                res = np.append(res, np.argmax(np.bincount(test)))
            return res

**IRIS**

В библиотеке scikit-learn есть несколько датасетов из коробки. Один из них [Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0)

In [3]:
iris = datasets.load_iris()

In [4]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [5]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [6]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [7]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)

assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

**Задание 2 (0.5 балла)**

Давайте попробуем добиться скорости работы на fit, predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. 

In [8]:
%time clf.fit(X_train, y_train)

Wall time: 1.99 ms


KNeighborsClassifier(algorithm='brute', n_neighbors=2)

In [9]:
%time my_clf.fit(X_train, y_train)

Wall time: 0 ns


In [10]:
%time clf.predict(X_test)

Wall time: 5.98 ms


array([1, 1, 2, 1, 0, 1, 2, 1, 0, 0, 1, 2, 0, 2, 0])

In [11]:
%time my_clf.predict(X_test)

Wall time: 1.99 ms


array([1, 1, 2, 1, 0, 1, 2, 1, 0, 0, 1, 2, 0, 2, 0], dtype=int64)

###### Задание 3 (1 балл)
Добавьте algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Необходимо добиться скорости работы на fit,  predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. Точность не должна уступать значению KNN из sklearn. 

In [12]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [13]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [14]:
%time clf.fit(X_train, y_train)

Wall time: 1.99 ms


KNeighborsClassifier(algorithm='kd_tree', n_neighbors=2)

In [15]:
%time my_clf.fit(X_train, y_train)

Wall time: 0 ns


In [16]:
%time clf.predict(X_test)

Wall time: 6.98 ms


array([0, 1, 0, 1, 1, 2, 0, 1, 2, 2, 0, 1, 1, 0, 2])

In [17]:
%time my_clf.predict(X_test)

Wall time: 11 ms


array([0, 1, 0, 1, 1, 2, 0, 1, 2, 2, 0, 1, 1, 0, 2], dtype=int64)

In [18]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"